In [ ]:
import pandas as pd
import numpy as np

def calculate_mag(x, y, z):
    return np.sqrt(x**2 + y**2 + z**2)

def classify_glucose(glucose, mean, std):
    if glucose > mean + std:
        return "PersHigh"
    elif glucose < mean - std:
        return "PersLow"
    else:
        return "PersNorm"

for id in range(1, len(dg_df)+1):
    if id == 7: # Skip ID 7
        continue
    fl_df = pd.read_csv(f'./big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/{id:03}/Food_Log_{id:03}.csv')
    acc_df = pd.read_csv(f'./big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/{id:03}/ACC_{id:03}.csv')
    eda_df = pd.read_csv(f'./big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/{id:03}/EDA_{id:03}.csv')
    bvp_df = pd.read_csv(f'./big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/{id:03}/BVP_{id:03}.csv')
    hr_df = pd.read_csv(f'./big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/{id:03}/HR_{id:03}.csv')
    ibi_df = pd.read_csv(f'./big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/{id:03}/IBI_{id:03}.csv')
    temp_df = pd.read_csv(f'./big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/{id:03}/TEMP_{id:03}.csv')
    dexcom_df = pd.read_csv(f'./big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/{id:03}/Dexcom_{id:03}.csv')

    biological_sex = dg_df.loc[dg_df['ID'] == id, 'Gender'].values[0]
    hbA1c = dg_df.loc[dg_df['ID'] == id, 'HbA1c'].values[0]

    print(f'ID : {id}')

    ## datetime 변환
    # Dexcom Gluecose
    dexcom_df.drop(dexcom_df.loc[:11].index, inplace=True)
    dexcom_df.rename(columns={'Timestamp (YYYY-MM-DDThh:mm:ss)': 'datetime'}, inplace=True)
    dexcom_df.rename(columns={'Glucose Value (mg/dL)': 'glucose'}, inplace=True)
    dexcom_df = dexcom_df[['datetime', 'glucose']]
    dexcom_df['datetime'] = pd.to_datetime(dexcom_df['datetime'])
    dexcom_df.set_index('datetime', inplace=True)

    # Food Log
    if id == 13:
        fl_df['time_begin'] = pd.to_datetime(fl_df['time_begin'], format='%m/%d/%Y %H:%M')
    else:
        fl_df['time_begin'] = pd.to_datetime(fl_df['time_begin'])
    fl_df.index = pd.DatetimeIndex(fl_df["time_begin"])
    fl_df = fl_df.sort_index()
    # Accelerometer
    acc_df['datetime'] = pd.to_datetime(acc_df['datetime'])
    acc_resampled = acc_df.set_index('datetime').resample('5Min').first().interpolate(method='time')
    # EDA
    eda_df['datetime'] = pd.to_datetime(eda_df['datetime'])
    eda_resampled = eda_df.set_index('datetime').resample('5Min').first().interpolate(method='time')
    # Heart Rate
    if id == 1:
        hr_df['datetime'] = pd.to_datetime(hr_df['datetime'], format='%m/%d/%y %H:%M')
    else:
        hr_df['datetime'] = pd.to_datetime(hr_df['datetime'])
    hr_resampled = hr_df.set_index('datetime').resample('5Min').first().interpolate(method='time')
    # IBI
    ibi_df['datetime'] = pd.to_datetime(ibi_df['datetime'])
    ibi_resampled = ibi_df.set_index('datetime').resample('5Min').first().interpolate(method='time')
    ibi_resampled[' ibi'] = ibi_resampled[' ibi'] * 1000  # 초 -> 밀리초 단위로
    # Temperature
    temp_df['datetime'] = pd.to_datetime(temp_df['datetime'])
    temp_resampled = temp_df.set_index('datetime').resample('5Min').first().interpolate(method='time')

    ## 날짜를 기준으로 데이터프레임을 분할하여 리스트에 저장
    daily_dfs = {'fl': [], 'acc': [], 'eda': [], 'hr': [], 'ibi': [], 'temp': [], 'bvp': [], 'dexcom': []}
    for name, group in dexcom_df.groupby(dexcom_df.index.date):
        daily_dfs['dexcom'].append(group)
    for name, group in fl_df.groupby(fl_df['time_begin'].dt.date):
        daily_dfs['fl'].append(group)
    for name, group in acc_resampled.groupby(acc_resampled.index.date):
        daily_dfs['acc'].append(group)
    for name, group in eda_resampled.groupby(eda_resampled.index.date):
        daily_dfs['eda'].append(group)
    for name, group in hr_resampled.groupby(hr_resampled.index.date):
        daily_dfs['hr'].append(group)
    for name, group in ibi_resampled.groupby(ibi_resampled.index.date):
        daily_dfs['ibi'].append(group)
    for name, group in temp_resampled.groupby(temp_resampled.index.date):
        daily_dfs['temp'].append(group)

    isok, daily_dfs = validate_daily_dfs(daily_dfs=daily_dfs)
    if isok == False:
        print(f'daily_dfs is modified...')
        
    base_df = pd.concat(daily_dfs['dexcom'])
    
    acc_df.drop(acc_df.index, inplace=True)
    eda_df.drop(eda_df.index, inplace=True)
    hr_df.drop(hr_df.index, inplace=True)
    ibi_df.drop(ibi_df.index, inplace=True)
    temp_df.drop(temp_df.index, inplace=True)

    if len(daily_dfs['acc']) > 0:
        acc_df = pd.concat(daily_dfs['acc'])
    if len(daily_dfs['eda']) > 0:
        eda_df = pd.concat(daily_dfs['eda'])
    if len(daily_dfs['hr']) > 0:   
        hr_df = pd.concat(daily_dfs['hr'])
    if len(daily_dfs['ibi']) > 0:
        ibi_df = pd.concat(daily_dfs['ibi'])
    if len(daily_dfs['temp']) > 0:
        temp_df = pd.concat(daily_dfs['temp'])

    # WakeTime, Minfrommid, Hourfrommid 계산
    hr_df['datetime'] = pd.to_datetime(hr_df['datetime'])
    acc_df['datetime'] = pd.to_datetime(acc_df['datetime'])
    hr_df['Minfrommid'] = hr_df['datetime'].dt.hour * 60 + hr_df['datetime'].dt.minute
    hr_df['Hourfrommid'] = hr_df['Minfrommid'] / 60
    acc_df['Minfrommid'] = acc_df['datetime'].dt.hour * 60 + acc_df['datetime'].dt.minute
    acc_df['Hourfrommid'] = acc_df['Minfrommid'] / 60

    # Historical averages 계산
    historical_hr_mean = hr_df[' hr'].mean()
    historical_hr_std = hr_df[' hr'].std()
    historical_acc_mean = acc_df[[' acc_x', ' acc_y', ' acc_z']].mean().mean()
    historical_acc_std = acc_df[[' acc_x', ' acc_y', ' acc_z']].std().mean()

    # 각 N 간격마다 평균 이하인지 체크
    hr_df['HR_Mean_Check'] = hr_df[' hr'] < historical_hr_mean
    hr_df['HR_Std_Check'] = hr_df[' hr'].rolling(window='10T').std() < historical_hr_std
    acc_df['ACC_Mean_Check'] = acc_df[[' acc_x', ' acc_y', ' acc_z']].mean(axis=1) < historical_acc_mean
    acc_df['ACC_Std_Check'] = acc_df[[' acc_x', ' acc_y', ' acc_z']].std(axis=1) < historical_acc_std

    # 포인트 계산
    hr_df['Points'] = hr_df[['HR_Mean_Check', 'HR_Std_Check']].sum(axis=1)
    acc_df['Points'] = acc_df[['ACC_Mean_Check', 'ACC_Std_Check']].sum(axis=1)

    # 3시간 롤링 평균
    hr_df['Rolling_Points'] = hr_df['Points'].rolling(window='3h').mean()
    acc_df['Rolling_Points'] = acc_df['Points'].rolling(window='3h').mean()

    # WakeTime 할당
    hr_df['WakeTime'] = (hr_df['Rolling_Points'] > 2).astype(int)
    acc_df['WakeTime'] = (acc_df['Rolling_Points'] > 2).astype(int)

    # WakeTime 조건 충족 확인
    hr_df['WakeTime_Final'] = (hr_df['WakeTime'].diff().fillna(0) == 1).astype(int)
    acc_df['WakeTime_Final'] = (acc_df['WakeTime'].diff().fillna(0) == 1).astype(int)

    # 윈도우 크기 설정
    window_size = '24h'

    # 각 타임스탬프에 대해 24시간 롤링 윈도우를 계산
    for timestamp in base_df.index:
        window_start = timestamp - pd.Timedelta(window_size)
        window_end = timestamp 
        
        if window_start < base_df.index[0]:
            continue

        # df_glucose의 롤링 윈도우 범위 내의 데이터 선택
        glucose_window = base_df.loc[window_start:window_end]
        # df_other의 동일한 범위 내의 데이터 선택
        fl_window = fl_df.loc[window_start:window_end - pd.Timedelta('5m')]
        acc_window = acc_df.loc[window_start:window_end - pd.Timedelta('5m')]
        eda_window = eda_df.loc[window_start:window_end - pd.Timedelta('5m')]
        hr_window = hr_df.loc[window_start:window_end - pd.Timedelta('5m')]
        ibi_window = ibi_df.loc[window_start:window_end - pd.Timedelta('5m')]
        temp_window = temp_df.loc[window_start:window_end - pd.Timedelta('5m')]

        ##### fl

        calories2hr = 0.
        protein2hr = 0.
        sugar2hr = 0.
        carbs2hr = 0.
        calories8hr = 0.
        protein8hr = 0.
        sugar8hr = 0.
        carbs8hr = 0.
        calories24hr = 0.
        protein24hr = 0.
        sugar24hr = 0.
        carbs24hr = 0.
        
        window2hr = fl_window[window_end - pd.Timedelta('2h') : window_end - pd.Timedelta('5m')]
        window8hr = fl_window[window_end - pd.Timedelta('8h') : window_end - pd.Timedelta('5m')]
        window24hr = fl_window[window_end - pd.Timedelta('24h') : window_end - pd.Timedelta('5m')]
        
        calories2hr = window2hr['calorie'].values.sum()
        protein2hr = window2hr['protein'].values.sum()
        sugar2hr = window2hr['sugar'].values.sum()
        carbs2hr = window2hr['total_carb'].values.sum()
        
        calories8hr = window8hr['calorie'].values.sum()
        protein8hr = window8hr['protein'].values.sum()
        sugar8hr = window8hr['sugar'].values.sum()
        carbs8hr = window8hr['total_carb'].values.sum()
        
        calories24hr = fl_window['calorie'].values.sum()
        protein24hr = fl_window['protein'].values.sum()
        sugar24hr = fl_window['sugar'].values.sum()
        carbs24hr = fl_window['total_carb'].values.sum()
        
        ##### acc
        acc_window = acc_window.copy()
        acc_window['vector_mag'] = calculate_mag(acc_window[' acc_x'], acc_window[' acc_y'], acc_window[' acc_z'])

        acc_mean = acc_window['vector_mag'].mean()
        acc_std = acc_window['vector_mag'].std()
        acc_min = acc_window['vector_mag'].min()
        acc_max = acc_window['vector_mag'].max()
        acc_q1g = acc_window['vector_mag'].quantile(0.25)
        acc_q3g = acc_window['vector_mag'].quantile(0.75)
        acc_skew = acc_window['vector_mag'].skew()

        ###### eda
        eda_mean = eda_window[' eda'].mean()
        eda_std = eda_window[' eda'].std()
        eda_min = eda_window[' eda'].min()
        eda_max = eda_window[' eda'].max()
        eda_q1g = eda_window[' eda'].quantile(0.25)
        eda_q3g = eda_window[' eda'].quantile(0.75)
        eda_skew = eda_window[' eda'].skew()

        # peak detection
        peaks_idx, properties = find_peaks(eda_window[' eda'], height=0, distance=4, prominence=0.3)
        peak_eda = len(peaks_idx)

        number_of_windows, peaks_in_window = find_peaks_in_rolling_window(eda_window, window_size='2h')
        peakEDA2hr_sum = peaks_in_window
        if number_of_windows > 0:
            peakEDA2hr_mean = peaks_in_window / number_of_windows
        else:
            print(f"ID : {id}, timestamp : {timestamp} -> number of 2h eda rolling window == 0")
            peakEDA2hr_mean = np.nan

        ###### hr

        hr_mean = hr_window[' hr'].mean()
        hr_std = hr_window[' hr'].std()
        hr_min = hr_window[' hr'].min()
        hr_max = hr_window[' hr'].max()
        hr_q1g = hr_window[' hr'].quantile(0.25)
        hr_q3g = hr_window[' hr'].quantile(0.75)
        hr_skew = hr_window[' hr'].skew()
        
        ###### ibi

        maxHRV = ibi_window[' ibi'].max()
        minHRV = ibi_window[' ibi'].min()
        medianHRV = ibi_window[' ibi'].median()
        meanHRV = ibi_window[' ibi'].mean()
        sdnn = ibi_window[' ibi'].std()

        nn50 = 0.
        pnn50 = np.nan
        rmssd = np.nan
        if len(ibi_window) > 0:
            for i in range(len(ibi_window)-1):
                if np.abs(ibi_window.iloc[i+1][' ibi'] - ibi_window.iloc[i][' ibi']) > 50:
                    nn50 +=1

            pnn50 = nn50/len(ibi_window)

            rmssd = 0.
            for i in range(len(ibi_window)-1):
                rmssd = np.abs(ibi_window.iloc[i+1][' ibi'] - ibi_window.iloc[i][' ibi'])**2
            rmssd = np.sqrt(rmssd) / len(ibi_window)
        else:
            pnn50 = np.nan
            rmssd = np.nan

        ###### temp

        temp_mean = temp_window[' temp'].mean()
        temp_std = temp_window[' temp'].std()
        temp_min = temp_window[' temp'].min()
        temp_max = temp_window[' temp'].max()
        temp_q1g = temp_window[' temp'].quantile(0.25)
        temp_q3g = temp_window[' temp'].quantile(0.75)
        temp_skew = temp_window[' temp'].skew()

        ##### results
        
        base_df.loc[window_end, 'datetime'] = window_end.strftime('%Y-%m-%d %H:%M:%S')
        base_df.loc[window_end, 'personalMean'] = glucose_window['glucose'].mean()
        base_df.loc[window_end, 'personalStd'] = glucose_window['glucose'].std()
        base_df.loc[window_end, 'ID'] = id
        base_df.loc[window_end, 'HbA1c'] = hbA1c
        base_df.loc[window_end, 'Biological Sex'] = biological_sex
        base_df.loc[window_end, 'calories2hr'] = round(calories2hr, 2)
        base_df.loc[window_end, 'protein2hr'] = round(protein2hr, 2)
        base_df.loc[window_end, 'sugar2hr'] = round(sugar2hr, 2)
        base_df.loc[window_end, 'carbs2hr'] = round(carbs2hr, 2)
        base_df.loc[window_end, 'calories8hr'] = round(calories8hr, 2)
        base_df.loc[window_end, 'protein8hr'] = round(protein8hr, 2)
        base_df.loc[window_end, 'sugar8hr'] = round(sugar8hr, 2)
        base_df.loc[window_end, 'carbs8hr'] = round(carbs8hr, 2)
        base_df.loc[window_end, 'calories24hr'] = round(calories24hr, 2)
        base_df.loc[window_end, 'protein24hr'] = round(protein24hr, 2)
        base_df.loc[window_end, 'sugar24hr'] = round(sugar24hr, 2)
        base_df.loc[window_end, 'carbs24hr'] = round(carbs24hr, 2)
        base_df.loc[window_end, 'acc_mean'] = round(acc_mean, 2)
        base_df.loc[window_end, 'acc_std'] = round(acc_std, 2)
        base_df.loc[window_end, 'acc_min'] = round(acc_min, 2)
        base_df.loc[window_end, 'acc_max'] = round(acc_max, 2)
        base_df.loc[window_end, 'acc_q1g'] = round(acc_q1g, 2)
        base_df.loc[window_end, 'acc_q3g'] = round(acc_q3g, 2)
        base_df.loc[window_end, 'acc_skew'] = round(acc_skew, 2)
        base_df.loc[window_end, 'eda_mean'] = round(eda_mean, 2)
        base_df.loc[window_end, 'eda_std'] = round(eda_std, 2)
        base_df.loc[window_end, 'eda_min'] = round(eda_min, 2)
        base_df.loc[window_end, 'eda_max'] = round(eda_max, 2)
        base_df.loc[window_end, 'eda_q1g'] = round(eda_q1g, 2)
        base_df.loc[window_end, 'eda_q3g'] = round(eda_q3g, 2)
        base_df.loc[window_end, 'eda_skew'] = round(eda_skew, 2)
        base_df.loc[window_end, 'PeakEDA'] = peak_eda
        base_df.loc[window_end, 'PeakEDA2hr_sum'] = round(peakEDA2hr_sum, 2)
        base_df.loc[window_end, 'PeakEDA2hr_mean'] = round(peakEDA2hr_mean, 2)
        base_df.loc[window_end, 'hr_mean'] = round(hr_mean, 2)
        base_df.loc[window_end, 'hr_std'] = round(hr_std, 2)
        base_df.loc[window_end, 'hr_min'] = round(hr_min, 2)
        base_df.loc[window_end, 'hr_max'] = round(hr_max, 2)
        base_df.loc[window_end, 'hr_q1g'] = round(hr_q1g, 2)
        base_df.loc[window_end, 'hr_q3g'] = round(hr_q3g, 2)
        base_df.loc[window_end, 'hr_skew'] = round(hr_skew, 2)
        base_df.loc[window_end, 'maxHRV'] = round(maxHRV, 2)
        base_df.loc[window_end, 'minHRV'] = round(minHRV, 2)
        base_df.loc[window_end, 'medianHRV'] = round(medianHRV, 2)
        base_df.loc[window_end, 'meanHRV'] = round(meanHRV, 2)
        base_df.loc[window_end, 'SDNN'] = round(sdnn, 2)
        base_df.loc[window_end, 'NN50'] = round(nn50, 2)
        base_df.loc[window_end, 'pNN50'] = round(pnn50, 2)
        base_df.loc[window_end, 'RMSSD'] = round(rmssd, 2)
        base_df.loc[window_end, 'temp_mean'] = round(temp_mean, 2)
        base_df.loc[window_end, 'temp_std'] = round(temp_std, 2)
        base_df.loc[window_end, 'temp_min'] = round(temp_min, 2)
        base_df.loc[window_end, 'temp_max'] = round(temp_max, 2)
        base_df.loc[window_end, 'temp_q1g'] = round(temp_q1g, 2)
        base_df.loc[window_end, 'temp_q3g'] = round(temp_q3g, 2)
        base_df.loc[window_end, 'temp_skew'] = round(temp_skew, 2)

    # 각 측정값을 PersHigh, PersLow, PersNorm으로 분류
    base_df['label'] = base_df.apply(
        lambda row: classify_glucose(row['glucose'], row['personalMean'], row['personalStd']), axis=1
    )

    base_df.drop(columns=['personalMean', 'personalStd'], inplace=True)

    df = pd.concat([df, base_df], ignore_index=True)
